![Logo 1](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech1.jpg)
<div class="alert alert-block alert-info">
<h1> Komputerowe wspomaganie tłumaczenia </h1>
<h2> 1. <i>Podstawowe techniki wspomagania tłumaczenia</i> [laboratoria]</h2> 
<h3>Rafał Jaworski (2021)</h3>
</div>

![Logo 2](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech2.jpg)

Podstawowy scenariusz - tłumacz otrzymuje **dokument** będący ciągiem **segmentów** w **języku źródłowym** . Zadaniem tłumacza jest przetłumaczenie wszystkich segmentów na **język docelowy**. Oznacza to, że został wykonany już szereg operacji technicznych, a w tym:
- przesłanie pliku do tłumaczenia
- ekstrakcja tekstu z dokumentu
- podział tekstu na segmenty.


Tłumacz tłumaczy jeden segment na raz. Widzi kilka poprzednich oraz kilka następnych segmentów, jednak jego uwaga powinna skupiać się na aktywnym, aktualnie tłumaczonym segmencie. Nie oznacza to jednak, że tłumacz jest pozbawiony pomocy. Ma możliwość korzystania z różnorakich technik **komputerowego wspomagania tłumaczenia**.

# Pamięć tłumaczeń

Najbardziej podstawową techniką wspomagania tłumaczenia jest **pamięć tłumaczeń**. Pamięć tłumaczeń stanowi zbiór wcześniej przetłumaczonych zdań:

In [4]:
translation_memory = [('Wciśnij przycisk Enter', 'Press the ENTER button'), 
                      ('Sprawdź ustawienia sieciowe', 'Check the network settings')]

Kiedy tłumacz pracuje nad kolejnym zdaniem, program automatycznie przeszukuje pamięć tłumaczeń. Jeśli okaże się, że zdanie zostało już przetłumaczone, automatycznie pobierane jest jego tłumaczenie z pamięci:

In [5]:
def tm_lookup(sentence):
    return [entry[1] for entry in translation_memory if entry[0] == sentence]

In [6]:
tm_lookup('Wciśnij przycisk Enter')

['Press the ENTER button']

W takim przypadku tłumaczenie z pamięci jest najczęściej automatycznie wstawiane jako tłumaczenie segmentu źródłowego i tłumacz nie musi się już nim martwić (co więcej, nie dostaje za takie tłumaczenie zapłaty...)

Tłumacz pracuje dalej i każde wykonane przez niego tłumaczenie trafia do pamięci tłumaczeń (oczywiście dzieje się to automatycznie):

In [7]:
translation_memory.append(('Drukarka jest wyłączona', 'The printer is switched off'))
translation_memory.append(('Wymagane ponowne uruchomienie komputera', 'System restart required'))
translation_memory.append(('Wciśnij przycisk Enter', 'Press the ENTER key'))

Z całą pewnością tłumacz stara się przełożyć jakąś komputerową instrukcję. Po wielu godzinach pracy trafia na znane sobie zdanie "Wciśnij przycisk Enter". Następuje przeszukanie pamięci tłumaczeń:

In [8]:
tm_lookup('Wciśnij przycisk Enter')

['Press the ENTER button', 'Press the ENTER key']

Jak widać, mamy w pamięci dwa tłumaczenia tego zdania. To tłumacz musi zadecydować, które tłumaczenie wybrać. O metodach wspomagania decyzji tłumacza w takich przypadkach będziemy jeszcze mówić.

Przeanalizujmy teraz następujący przypadek: do tłumaczenia trafia zdanie: "Wciśnij przycisk ENTER":

In [9]:
tm_lookup('Wciśnij przycisk ENTER')

[]

Tutaj tłumacz nie ma szczęścia - restrykcyjny mechanizm wyszukiwania nie podaje mu podpowiedzi. Możemy jednak sprawić, żeby przeszukiwanie pamięci tłumaczeń nie brało pod uwagę wielkości liter.

### Ćwiczenie 1: zmodyfikuj funkcję tm_lookup w taki sposób, aby nie brała pod uwagę wielkości liter. Przetestuj zmodyfikowaną funkcję.

In [10]:
def tm_lookup(sentence):
    return [entry[1] for entry in translation_memory if entry[0].lower() == sentence.lower()]

Nasz tłumacz powinien być teraz zadowolony. Jednak w dalszej części dokumentu pojawiło się inne podobne zdanie, dla którego nie było tłumaczenia:

In [11]:
tm_lookup('Wciśnij przycisk [ENTER]')

[]

### Ćwiczenie 2: zmodyfikuj funkcję tm_lookup w taki sposób, aby nie brała pod uwagę znaków interpunkcyjnych. Rada - zdefiniuj funkcję sentence_similar.

In [20]:
import re

def sentence_similar(x):
    return re.sub(r'[^\w\s]', '', x).lower()

def tm_lookup(sentence):
    sentence_cleaned = sentence_similar(sentence)
    return [entry[1] for entry in translation_memory if sentence_similar(entry[0]) == sentence_cleaned]

# Test
print(tm_lookup('Wciśnij przycisk [ENTER]'))

    

['Press the ENTER button', 'Press the ENTER key']


Praca tłumacza oczywiście trwa nadal. Spróbujmy teraz zaradzić następującej sytuacji - tłumacz otrzymuje do tłumaczenia następujące zdanie:

In [13]:
tm_lookup('Wymagane ponowne uruchomienie maszyny')

[]

Znów nie otrzymuje od nas podpowiedzi i znów nic w tym dziwnego - w pamięci tłumaczeń nie znajduje się takie zdanie. Jest jednak zdanie podobne, różniące się jednym słowem.

### Ćwiczenie 3: zmodyfikuj funkcję tm_lookup w taki sposób, aby zwracała podpowiedzi także wtedy, gdy zdania różnią się tylko jednym słowem.

In [25]:
def sentence_similarity(sentence1, sentence2):
    words1 = set(sentence_similar(sentence1).split())
    words2 = set(sentence_similar(sentence2).split())
    common_words = words1.intersection(words2)
    if len(common_words) == len(words1) - 1:
        return True
    else:
        return False

def tm_lookup(sentence):
    sentence_cleaned = sentence_similar(sentence)
    result = []
    for entry in translation_memory: 
        if sentence_similarity(entry[0], sentence_cleaned):
            result.append(entry[1])
    if len(result) == 0:
        result.append('Brak podpowiedzi')
    return result

# Test
print(tm_lookup('Wymagane ponowne uruchomienie maszyny'))


['System restart required']


# Słownik kontekstowy / glosariusz

Inną podstawową pomocą dla naszego tłumacza jest słownik, zwany także glosariuszem. Stanowi on zbiór terminów w języku źródłowym wraz z ich tłumaczeniami na język docelowy:

In [15]:
glossary = [('komputer', 'computer'), ('przycisk', 'button'), ('drukarka', 'printer')]

Podczas tłumaczenia, glosariusz jest przeszukiwany w celu znalezienia aktualnie tłumaczonych słów.

In [16]:
def glossary_lookup(sentence):
    sentence_words = sentence.split()
    return [entry for entry in glossary if entry[0] in sentence_words]

In [17]:
glossary_lookup('Każda drukarka posiada przycisk wznowienia drukowania')

[('przycisk', 'button'), ('drukarka', 'printer')]

### Ćwiczenie 4: (tym razem teoretyczne) Jaka jest złożoność obliczeniowa czasowa przedstawionego powyżej algorytmu względem liczby haseł w słowniku (n) oraz liczby słów w zdaniu do tłumaczenia (m)?

Odpowiedź: Złożoność czasowa algorytmu glossary_lookup zależy od dwóch czynników: liczby haseł w słowniku (n) oraz liczby słów w zdaniu (m).

Dla każdego słowa w zdaniu (m), algorytm musi sprawdzić, czy dane słowo znajduje się w słowniku (n). Sprawdzenie to wymaga przeszukania całego słownika, co daje nam złożoność O(n) dla jednego słowa.

Ponieważ musimy wykonać to przeszukiwanie dla każdego z m słów w zdaniu, całkowita złożoność algorytmu to O(m * n).

### Ćwiczenie 5: zmodyfikuj funkcję glossary_lookup w taki sposób, aby nie brała pod uwagę wielkości liter.

In [18]:
def glossary_lookup(sentence):
    sentence_words = sentence.lower().split()
    return [entry for entry in glossary if entry[0].lower() in sentence_words]

# Test
glossary = [('komputer', 'computer'), ('przycisk', 'button'), ('drukarka', 'printer')]
print(glossary_lookup('Każda drukarka posiada przycisk wznowienia drukowania'))  # Powinno zwrócić [ ('drukarka', 'printer'), ('przycisk', 'button') ]

[('przycisk', 'button'), ('drukarka', 'printer')]


### Ćwiczenie 6: zmodyfikuj funkcję glossary_lookup w taki sposób, aby jej złożoność obliczeniowa była mniejsza.

In [23]:
def glossary_lookup(sentence):
    sentence_words = set(sentence.lower().split())
    glossary_dict = {entry[0].lower(): entry[1] for entry in glossary}
    return [(word, glossary_dict[word]) for word in sentence_words if word in glossary_dict]

# Test
print(glossary_lookup('Każda drukarka posiada przycisk wznowienia drukowania'))


[('drukarka', 'printer'), ('przycisk', 'button')]
